In [3]:
!pip install openai==0.28

In [5]:
import openai
from tqdm.auto import tqdm
import time


api_key="923a50a26293c4b70d4ba05e4642b5ca",
base_url="https://chat-ai.academiccloud.de/v1"


model = "qwen3-32b"
temperature = 0.8

def generate_crisis_post(place, crisis_event, lat, lon, Amenities):
    prompt = f"""
    You are a Hamburg resident live-posting on social media.

    A **{crisis_event}** is unfolding near **{place}**. There are several amenities: {Amenities} in this area — please make sure your description stays **within the vicinity of this location**, based on these surroundings.

    Write a short and urgent **first-person** social media post in **English**, consisting of **3 to 4 vivid, emotional sentences**.
    Make sure a **clear reference to the location** in the style of German addresses (but don't include the structure directly). Focus on urgency, chaos, atmosphere, and personal reaction.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    return response.choices[0].message.content.strip()


def batch_generate_crisis_posts(df):
    posts = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating posts"):
        try:
            post = generate_crisis_post(
                place=row["Place"],
                crisis_event=row["Crisis_Event"],
                lat=row["Latitude"],
                lon=row["Longitude"],
                Amenities=row["Amenities"]
            )
        except Exception as e:
            print(f"⚠️ Error at row {_}: {e}")
            post = ""
        posts.append(post)
        time.sleep(0.4)
    df["Crisis_Post"] = posts
    return df
